In [44]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KDTree
from sklearn.svm import SVR
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
import math
from sklearn.preprocessing import LabelEncoder
import pickle

In [19]:
df = pd.read_csv("65d4f0fcb8af9_amex_campus_challenge_train_3.csv")

In [33]:
unique_names = df.customer.unique()
customer_rates = {elem : pd.DataFrame() for elem in unique_names}
for customer, df_customer in df.groupby('customer'):
    recommended = not_recommended = activated_recommended = activated_not_recommended = not_activated_recommended = not_activated_not_recommended = 0
    for index, row in df_customer.iterrows():
        if row['activation'] == 1 and row['ind_recommended'] == 1:
            recommended += 1
            activated_recommended += 1
        elif row['activation'] == 1 and row['ind_recommended'] == 0:
            not_recommended += 1
            activated_not_recommended += 1
        elif row['activation'] == 0 and row['ind_recommended'] == 1:
            recommended += 1
            not_activated_recommended += 1
        elif row['activation'] == 0 and row['ind_recommended'] == 0:
            not_recommended += 1
            not_activated_not_recommended += 1
    if(recommended == 0 and not_recommended == 0):
        recommended_activation = organic_activation = recommended_non_activation = organic_non_activation = 0.0
    elif(recommended == 0):
        recommended_activation = recommended_non_activation = 0.0
        organic_activation = activated_not_recommended / not_recommended
        organic_non_activation = not_activated_not_recommended / not_recommended
    elif(not_recommended == 0):
        organic_activation = organic_non_activation = 0.0
        recommended_activation = activated_recommended / recommended
        recommended_non_activation = not_activated_recommended / recommended
    else:
        recommended_activation = activated_recommended / recommended
        organic_activation = activated_not_recommended / not_recommended
        recommended_non_activation = not_activated_recommended / recommended
        organic_non_activation = not_activated_not_recommended / not_recommended
        
    incremental_activation = recommended_activation - organic_activation
    customer_rates[customer] = pd.DataFrame({'recommended_activation': recommended_activation, 'organic_activation': organic_activation, 'recommended_non_activation': recommended_non_activation, 'organic_non_activation': organic_non_activation, 'incremental_activation': incremental_activation}, index=[0])

In [45]:
with open('customer_rates.pkl', 'wb+') as f:
    pickle.dump(customer_rates, f)

In [46]:
with open('customer_rates.pkl', 'rb') as f:
    customer_rates = pickle.load(f)

In [38]:
def set_incremental_rates(customer):
    return customer_rates[customer]['incremental_activation']


In [39]:
df['incremental_activation'] = df['customer'].apply(set_incremental_rates)

In [50]:
df = df.fillna(df.mean())

In [51]:
df.loc[df['customer'] == 103370]

,ind_recommended,activation,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,...,merchant_profile_03,customer_digital_activity_01,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,customer,merchant,incremental_activation
797,1,0,6.000000,134.701900,22.137910,87.000000,3918.410000,177.000000,127.000000,30.0,...,95401.000000,0.488372,27.022500,76.804926,5.333333,266.0,1.698412,103370,375347,0.166667
1122243,1,0,9.745803,114.122727,22.166860,84.000000,3812.700000,172.000000,122.000000,33.0,...,88333.000000,0.488372,23.300000,76.804926,5.333333,266.0,1.572730,103370,242435,0.166667
1243840,0,0,9.745803,134.701900,90.934000,5.000000,454.670000,5.000000,5.000000,30.0,...,31234.000000,0.488372,49.642857,76.804926,5.333333,266.0,3.033815,103370,305440,0.166667
2261963,0,0,9.745803,134.701900,22.137910,87.000000,3918.410000,177.000000,127.000000,30.0,...,85812.000000,0.488372,16.157534,76.804926,5.333333,266.0,1.357209,103370,249649,0.166667
3128689,0,0,7.000000,114.122727,14.980000,2.000000,29.960000,2.000000,2.000000,33.0,...,8613.000000,0.488372,40.252500,76.804926,5.333333,266.0,3.102754,103370,558726,0.166667
3573988,0,0,9.745803,114.122727,90.934000,5.000000,454.670000,5.000000,5.000000,33.0,...,34726.000000,0.488372,102.600000,76.804926,5.333333,266.0,1.450614,103370,233406,0.166667
3628486,0,0,9.745803,148.496047,46.136667,2.000000,138.410000,3.000000,2.000000,24.0,...,9169.000000,0.488372,44.346667,76.804926,5.333333,266.0,1.566149,103370,271338,0.166667
5507908,0,0,9.745803,134.701900,116.329031,12.486146,2160.901083,30.912555,23.552949,30.0,...,1949.000000,0.488372,166.525000,76.804926,5.333333,266.0,1.683341,103370,56855,0.166667
5526281,1,0,9.745803,148.496047,90.934000,5.000000,454.670000,5.000000,5.000000,24.0,...,33524.000000,0.488372,54.060000,76.804926,5.333333,266.0,1.450614,103370,545119,0.166667
6539567,1,0,9.745803,114.122727,116.329031,12.486146,2160.901083,30.912555,23.552949,33.0,...,1770.000000,0.488372,1425.000000,76.804926,5.333333,266.0,2.960558,103370,95727,0.166667
